In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
pd.set_option('display.max_columns', None) # to see all the columns on my dataframe
listing= pd.read_csv('../data/raw_data/listing.csv',low_memory=False)
calendar = pd.read_csv('../data/raw_data/calendar.csv')
review =pd.read_csv('../data/raw_data/reviews 2.csv')

In [5]:
listing.drop(['scrape_id','host_thumbnail_url','host_picture_url','host_listings_count',
              'host_verifications','host_identity_verified','neighbourhood','neighbourhood_cleansed',
             'neighbourhood_group_cleansed','bathrooms','availability_30','availability_60','availability_90','availability_365',
             'calendar_last_scraped','number_of_reviews_ltm','number_of_reviews_l30d','license','instant_bookable',
             'calculated_host_listings_count','calculated_host_listings_count_entire_homes','calculated_host_listings_count_private_rooms',
             'calculated_host_listings_count_shared_rooms','host_response_time','host_response_rate','host_acceptance_rate','minimum_minimum_nights',
             'maximum_minimum_nights','minimum_maximum_nights','maximum_maximum_nights','minimum_nights_avg_ntm','maximum_nights_avg_ntm',
             'calendar_updated','first_review','last_review'],axis=1,inplace=True)

calendar.drop('adjusted_price',axis=1,inplace=True)

In [15]:
print('listings shape: ',listing.shape)
print('calendar shape: ', calendar.shape)
print('review shape: ', review.shape)

listings shape:  (37713, 39)
calendar shape:  (13758434, 6)
review shape:  (876200, 6)


In [39]:
listing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37713 entries, 0 to 37712
Data columns (total 39 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   id                           37713 non-null  int64         
 1   listing_url                  37713 non-null  object        
 2   last_scraped                 37713 non-null  datetime64[ns]
 3   name                         37713 non-null  object        
 4   description                  37713 non-null  object        
 5   neighborhood_overview        37713 non-null  object        
 6   picture_url                  37713 non-null  object        
 7   host_id                      37713 non-null  int64         
 8   host_url                     37713 non-null  object        
 9   host_name                    37713 non-null  object        
 10  host_since                   37713 non-null  datetime64[ns]
 11  host_location                37713 non-nu

In [17]:
calendar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13758434 entries, 0 to 13758433
Data columns (total 6 columns):
 #   Column          Dtype  
---  ------          -----  
 0   listing_id      int64  
 1   date            object 
 2   available       object 
 3   price           object 
 4   minimum_nights  float64
 5   maximum_nights  float64
dtypes: float64(2), int64(1), object(3)
memory usage: 629.8+ MB


In [18]:
review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 876200 entries, 0 to 876199
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   listing_id     876200 non-null  int64 
 1   id             876200 non-null  int64 
 2   date           876200 non-null  object
 3   reviewer_id    876200 non-null  int64 
 4   reviewer_name  876194 non-null  object
 5   comments       875334 non-null  object
dtypes: int64(3), object(3)
memory usage: 40.1+ MB


In [24]:
listing.isnull().sum()

id                                0
listing_url                       0
last_scraped                      0
name                              0
description                       0
neighborhood_overview             0
picture_url                       0
host_id                           0
host_url                          0
host_name                         0
host_since                        0
host_location                     0
host_about                        0
host_is_superhost                 0
host_neighbourhood                0
host_total_listings_count         0
host_has_profile_pic              0
latitude                          0
longitude                         0
property_type                     0
room_type                         0
accommodates                      0
bathrooms_text                    0
bedrooms                          0
beds                              0
amenities                         0
price                             0
minimum_nights              

In [30]:
calendar.isnull().sum()

listing_id        0
date              0
available         0
price             0
minimum_nights    0
maximum_nights    0
dtype: int64

In [31]:
review.isnull().sum()

listing_id       0
id               0
date             0
reviewer_id      0
reviewer_name    0
comments         0
dtype: int64

In [23]:
listing['name'].fillna(str(listing['id']), inplace=True)
listing['description'].fillna('None', inplace=True)
listing['neighborhood_overview'].fillna('None', inplace=True)
listing['host_name'].fillna(str(listing['id']), inplace=True)
listing['host_since'].fillna(listing['host_since'].mode()[0], inplace=True)
listing['host_location'].fillna('New York, New York, United States', inplace=True)
listing['host_about'].fillna('None', inplace=True)
listing['host_is_superhost'].fillna('f', inplace=True)
listing['host_neighbourhood'].fillna('None', inplace=True)
listing['host_total_listings_count'].fillna('0', inplace=True)
listing['host_has_profile_pic'].fillna('f', inplace=True)
listing['bathrooms_text'].fillna('1 bath', inplace=True)
listing['bedrooms'].fillna('1.0', inplace=True)
listing['beds'].fillna(listing['beds'].mode()[0], inplace=True)

calendar.dropna(subset=['price'],inplace=True)
calendar.dropna(subset=['minimum_nights'],inplace=True)

review['reviewer_name'].fillna('None', inplace=True)
review['comments'].fillna('None', inplace=True)

In [32]:
listing['price'] = listing['price'].replace('[\$\,\.]', '', regex=True).astype(int)
calendar['price'] = calendar['price'].replace('[\$\,\.]', '', regex=True).astype(int)

#https://stackoverflow.com/questions/44469313/price-column-object-to-int-in-pandas

In [37]:
# Change date column to be datetime dtype
listing['last_scraped'] = pd.to_datetime(listing['last_scraped'])
listing['host_since'] = pd.to_datetime(listing['host_since'])

review['date'] = pd.to_datetime(review['date'])

calendar['date'] = pd.to_datetime(calendar['date'])


In [38]:
listing.head()

,id,listing_url,last_scraped,name,description,neighborhood_overview,picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_is_superhost,host_neighbourhood,host_total_listings_count,host_has_profile_pic,latitude,longitude,property_type,room_type,accommodates,bathrooms_text,bedrooms,beds,amenities,price,minimum_nights,maximum_nights,has_availability,number_of_reviews,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month
0,2595,https://www.airbnb.com/rooms/2595,2021-11-03,Skylit Midtown Castle,"Beautiful, spacious skylit studio in the heart...",Centrally located in the heart of Manhattan ju...,https://a0.muscache.com/pictures/f0813a11-40b2...,2845,https://www.airbnb.com/users/show/2845,Jennifer,2008-09-09,"New York, New York, United States",A New Yorker since 2000! My passion is creatin...,f,Midtown,8.0,t,40.75356,-73.98559,Entire rental unit,Entire home/apt,1,1 bath,1.0,1.0,"[""Bathtub"", ""Dedicated workspace"", ""Hair dryer...",15000,30,1125,t,48,4.70,4.72,4.62,4.76,4.79,4.86,4.41,0.33
1,3831,https://www.airbnb.com/rooms/3831,2021-11-03,"Whole flr w/private bdrm, bath & kitchen(pls r...","Enjoy 500 s.f. top floor in 1899 brownstone, w...",Just the right mix of urban center and local n...,https://a0.muscache.com/pictures/e49999c2-9fd5...,4869,https://www.airbnb.com/users/show/4869,LisaRoxanne,2008-12-07,"New York, New York, United States",Laid-back Native New Yorker (formerly bi-coast...,f,Clinton Hill,1.0,t,40.68494,-73.95765,Entire guest suite,Entire home/apt,3,1 bath,1.0,3.0,"[""Microwave"", ""Oven"", ""Pack \u2019n play/Trave...",7300,1,730,t,408,4.46,4.59,4.50,4.79,4.81,4.72,4.65,4.91
2,5121,https://www.airbnb.com/rooms/5121,2021-11-03,BlissArtsSpace!,<b>The space</b><br />HELLO EVERYONE AND THANK...,None,https://a0.muscache.com/pictures/2090980c-b68e...,7356,https://www.airbnb.com/users/show/7356,Garon,2009-02-03,"New York, New York, United States","I am an artist(painter, filmmaker) and curato...",f,Bedford-Stuyvesant,1.0,t,40.68535,-73.95512,Private room in rental unit,Private room,2,1 bath,1.0,1.0,"[""Long term stays allowed"", ""Heating"", ""Wifi"",...",6000,30,730,t,50,4.52,4.22,4.09,4.91,4.91,4.47,4.52,0.53
3,5136,https://www.airbnb.com/rooms/5136,2021-11-03,"Spacious Brooklyn Duplex, Patio + Garden",We welcome you to stay in our lovely 2 br dupl...,None,https://a0.muscache.com/pictures/miso/Hosting-...,7378,https://www.airbnb.com/users/show/7378,Rebecca,2009-02-03,"Brooklyn, New York, United States","Rebecca is an artist/designer, and Henoch is i...",f,Greenwood Heights,1.0,t,40.66265,-73.99454,Entire rental unit,Entire home/apt,4,1.5 baths,2.0,2.0,"[""BBQ grill"", ""Dryer"", ""Carbon monoxide alarm""...",27500,5,1125,t,2,5.00,5.00,5.00,5.00,5.00,4.50,5.00,0.02
4,5178,https://www.airbnb.com/rooms/5178,2021-11-03,Large Furnished Room Near B'way,Please don’t expect the luxury here just a bas...,"Theater district, many restaurants around here.",https://a0.muscache.com/pictures/12065/f070997...,8967,https://www.airbnb.com/users/show/8967,Shunichi,2009-03-03,"New York, New York, United States",I used to work for a financial industry but no...,f,Hell's Kitchen,1.0,t,40.76457,-73.98317,Private room in rental unit,Private room,2,1 bath,1.0,1.0,"[""Conditioner"", ""Drying rack for clothing"", ""H...",6800,2,14,t,505,4.21,4.20,3.73,4.66,4.41,4.87,4.35,3.70


In [33]:
listing.describe()

,id,host_id,latitude,longitude,accommodates,beds,price,minimum_nights,maximum_nights,number_of_reviews,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month
count,3.771300e+04,3.771300e+04,37713.000000,37713.000000,37713.000000,37713.000000,37713.000000,37713.000000,3.771300e+04,37713.000000,28500.000000,27879.000000,27890.000000,27872.000000,27883.000000,27868.000000,27868.000000,28501.000000
mean,2.883977e+07,1.117833e+08,40.729186,-73.949361,2.784981,1.555511,16542.255986,22.130645,5.866011e+04,23.233368,4.584785,4.746614,4.613382,4.816832,4.811570,4.752293,4.650597,1.640820
std,1.710942e+07,1.272200e+08,0.055662,0.050442,1.863980,1.054040,29236.618160,30.809556,1.105915e+07,50.153962,0.839473,0.491423,0.572011,0.435073,0.456708,0.412702,0.511760,4.220807
min,2.595000e+03,2.438000e+03,40.504559,-74.249840,0.000000,1.000000,0.000000,1.000000,1.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000
25%,1.309845e+07,1.110595e+07,40.689380,-73.983620,2.000000,1.000000,6900.000000,3.000000,6.000000e+01,1.000000,4.570000,4.710000,4.500000,4.810000,4.820000,4.670000,4.560000,0.120000
50%,2.996451e+07,4.886606e+07,40.725160,-73.955020,2.000000,1.000000,11000.000000,30.000000,1.125000e+03,4.000000,4.820000,4.900000,4.800000,4.960000,4.970000,4.880000,4.780000,0.470000
75%,4.547279e+07,1.918493e+08,40.762510,-73.930950,4.000000,2.000000,18000.000000,30.000000,1.125000e+03,20.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,1.690000
max,5.312763e+07,4.298401e+08,40.914070,-73.710870,16.000000,24.000000,1000000.000000,1250.000000,2.147484e+09,1009.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,136.000000


In [40]:
listing.to_csv('../data/listings_clean.csv', index=False)
review.to_csv('../data/review_clean.csv', index=False)
calendar.to_csv('../data/calendar_clean_clean.csv', index=False)

KeyboardInterrupt: 